SN Photometric Challenge Parser

### First, an example of SNANASims dealing with SNANA FITS file

In [ ]:
import sndata as snd
from sndata import SNANASims
from astropy.table import Table
from matplotlib import pyplot as plt
megacamBandNames = 'ugriz'
#megacamRegisteredNames = tuple('megacam' + band for band in megacamBandNames)
megacamRegisteredNames = tuple('megacam' + band for band in megacamBandNames)

In [ ]:
snanafile = SNANASims.fromSNANAfileroot(snanafileroot='snana_fits',
                                       location='../sndata/example_data/',
                                       coerce_inds2int=False,
                                       SNANABandNames=megacamBandNames,
                                       registeredBandNames=megacamRegisteredNames)

In [ ]:
#print(snanafile.photFile)
#print(snanafile.headFile)
snanafile.headData
#print(snanafile.phot)
#print(snanafile.bandNames)
#print(snanafile.newbandNames)
#print(snanafile.bandNameDict)


In [ ]:
lcInstance = snanafile.get_SNANA_photometry(snid='03D1aw')


In [ ]:
lcInstance.lightCurve[['mjd', 'band', 'flux', 'fluxerr', 'zp', 'zpsys']]


In [ ]:
lcInstance.bandNameDict

In [ ]:
snanalc = snd.LightCurve(lcInstance.lightCurve[['mjd', 'band', 'flux', 'fluxerr', 'zp', 'zpsys']])


In [ ]:
snanalc

### Second, check if SNANASims is able to read old SNANA ASCII files from SN Challenge 2010

In [1]:
import sndata as snd
from sndata import SNChalSims
from astropy.table import Table

megacamBandNames = 'ugriz'
#megacamRegisteredNames = tuple('megacam' + band for band in megacamBandNames)
megacamRegisteredNames = tuple('megacam' + band for band in megacamBandNames)

In [ ]:
SNChalSims.

In [2]:
charfile = SNChalSims.fromSNChal(snanafileroot='DES_BLIND+HOSTZ.tar.gz', zipped=True,
                                       location='../sndata/example_data/',
                                       SNANABandNames=megacamBandNames,
                                       registeredBandNames=megacamRegisteredNames)

0 DES_BLIND+HOSTZ/                                                                                    0000775 0005415 0012504 00000000000 11360736261 012704  5                                                                                                    ustar   rkessler                        sdss                                                                                                                                                                                                                   DES_BLIND+HOSTZ/DES_SN004216.DAT                                                                    0000664 0005415 0012504 00000013340 11360734453 014730  0                                                                                                    ustar   rkessler                        sdss                                                                                                                                                                                         

In [8]:
charfile.headData
print(charfile.photFile)
print(charfile.headFile)
print(charfile.phot)
print(charfile.bandNames)
print(charfile.newbandNames)
#print(snanafile.bandNameDict)

None
None
None
ugriz
('megacamu', 'megacamg', 'megacamr', 'megacami', 'megacamz')


In [ ]:
charzip = SNANASims.SNChalParser(snanafileroot='DES_BLIND+HOSTZ.tar.gz',
                                       location='../sndata/example_data/',
                                       SNANABandNames=megacamBandNames,
                                       registeredBandNames=megacamRegisteredNames,
                                       zipped=True)

In [ ]:
from __future__ import absolute_import, print_function, division
import fitsio
import pandas as pd
import numpy as np
import os
import gzip
from astropy.table import Table, Column

In [ ]:
location = os.path.abspath('../sndata/example_data/')
snanafileroot='DES_BLIND+HOSTZ.tar.gz'
#snanafileroot='DES_SN000041.DAT'
filename = os.path.join(location, snanafileroot)
zipped=True
#zipped=False

In [ ]:

if zipped:
    operator = gzip.open
    lcoperator = gzip.GzipFile
else:
    operator = open
    lcoperator = str


In [ ]:
with operator(filename, 'rb') as f:
    for lineno,line in enumerate(f):
        print(lineno,line)

In [ ]:
with operator(filename, 'rb') as f:
    print(f)
    for lineno,line in enumerate(f):
        print(lineno,line)
        s = line.split(':')
        if len(s) <= 0:
            continue
        #if s[0] == 'SURVEY':
        #    survey = s[1].strip()
        elif s[0] == 'SNID':
            snid = int(s[1].strip())
        elif s[0] == 'IAUC':
            iauc = s[1].strip()
        elif s[0] == 'SNTYPE':
            sn_type = int(s[1].strip())
        elif s[0] == 'FILTERS':
            filters = s[1].strip()
        elif s[0] == 'RA':
            ra = float(s[1].split('deg')[0].strip())
        elif s[0] == 'DECL':
            decl = float(s[1].split('deg')[0].strip())
        elif s[0] == 'FAKE':
            fake = int(s[1].split('(')[0].strip())
        elif s[0] == 'MWEBV':
            mwebv = float(s[1].split('MW')[0].strip())
        elif s[0] == 'REDSHIFT_SPEC':
            spec = float(s[1].split('+-')[0].strip()), float(s[1].split('+-')[1]    .strip())
        elif s[0] == 'HOST_GALAXY_GALID':
            hostid = int(s[1].strip())
        elif s[0] == 'HOST_GALAXY_PHOTO-Z':
            hostz = float(s[1].split('+-')[0].strip()), float(s[1].split('+-')[1    ].strip())
        elif s[0] == 'NOBS':
            nobs = int(s[1].strip())
        elif s[0] == 'VARLIST':
            break

In [ ]:
lightcurve = pd.read_table(lcoperator(filename), sep='\s+', header=0, 
                    skiprows= lineno, skipfooter=1,
                    comment='DETECTION', engine='python',
                    usecols=[1,2,4,5])

In [ ]:
#lightcurve['SNR'] = lightcurve['FLUXCAL']/lightcurve['FLUXCALERR']
#lightcurve['MAG'] = -2.5*np.log10(lightcurve['FLUXCAL'])+27.5
#lightcurve['MAGERR'] = 2.5*np.log10(lightcurve['FLUXCALERR'])
lightcurve['ZP'] = [27.5 for i in range(len(lightcurve['FLUXCAL']))]
lightcurve['ZPSYS'] = ['ab' for i in range(len(lightcurve['FLUXCAL']))]

In [ ]:
#lightcurve = lightcurve.drop('FIELD', 1)
lightcurve


In [ ]:
snid=np.array

In [ ]:
params = {'snid': snid,'iauc': iauc,'fake': fake,'ra': ra,'decl': decl,'pixsize': -9.0,
                  'nxpix': 0,'nypix': -9,'sn_type': sn_type,'nobs': nobs, 'ptrobs_min': -9,'ptrobs_max': -9,                   
                  'mwebv': mwebv, 'mwebv_err': -9, 'redshift_helio': -9,'redshift_helio_err': -9,
                  'redshift_final': hostz[0],'redshift_final_err': hostz[1],'hostid': hostid,
                  'hostgal_photoz': hostz[0],'hostgal_photoz_err': hostz[1],'hostgal_specz': spec[0],'hostgal_specz_err': spec[1],
                  'hostgal_snsep': -9.0,'hostgal_logmass': -9.0,'hostgal_logmass_err': -9.0,
                  'hostgal_mag_g': -9.0,'hostgal_mag_r': -9.0,'hostgal_mag_i': -9.0,'hostgal_mag_z': -9.0,
                  'hostgal_sb_fluxcal_g': -9.0,'hostgal_sb_fluxcal_r': -9.0,'hostgal_sb_fluxcal_i': -9.0,'hostgal_sb_fluxcal_z': -9.0,
                  'peakmjd': -9.0,'search_type': -9}
        

In [ ]:
head=pd.DataFrame.from_dict({k:np.array([v]) for k, v in params.iteritems()})

In [ ]:
head

In [ ]:
from collections import defaultdict

In [ ]:
a=defaultdict(list)

In [ ]:
a['snid'].append(1)

In [ ]:
a

In [ ]:
t=lightcurve[lightcurve.FLT =='i']
plt.errorbar(lightcurve[lightcurve.FLT =='g']['MJD'],lightcurve[lightcurve.FLT =='g']['MAG'],yerr=lightcurve[lightcurve.FLT =='g']['MAGERR'], capsize=0,fmt='--o')
plt.errorbar(lightcurve[lightcurve.FLT =='r']['MJD'],lightcurve[lightcurve.FLT =='r']['MAG'],yerr=lightcurve[lightcurve.FLT =='r']['MAGERR'], capsize=0,fmt='--o')
plt.errorbar(lightcurve[lightcurve.FLT =='i']['MJD'],lightcurve[lightcurve.FLT =='i']['MAG'],yerr=lightcurve[lightcurve.FLT =='i']['MAGERR'], capsize=0,fmt='--o')
plt.errorbar(lightcurve[lightcurve.FLT =='z']['MJD'],lightcurve[lightcurve.FLT =='z']['MAG'],yerr=lightcurve[lightcurve.FLT =='z']['MAGERR'], capsize=0,fmt='--o')
plt.gca().invert_yaxis()
plt.show()